In [1]:
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Add, Dense, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,
    AveragePooling2D, MaxPooling2D, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Layer
from sklearn.metrics import precision_recall_fscore_support
from google.colab import drive

drive.mount('/content/drive')

# Load Fashion-MNIST
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
y_train = y_train.ravel()
y_test = y_test.ravel()

# Custom smooth activation
class TanhReLU(Layer):
    def call(self, inputs):
        return tf.where(inputs < 0, tf.tanh(inputs), inputs)

# ResNet-18 architecture
class ResNet18:
    def __init__(self, num_classes=10, input_shape=(28, 28, 1)):
        self.num_classes = num_classes
        self.input_shape = input_shape
        self.activation_layer = TanhReLU()

    def identity_block18(self, x, filters):
        x_skip = x
        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x = self.activation_layer(x)

        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)

        x = Add()([x, x_skip])
        x = self.activation_layer(x)
        return x

    def convolutional_block18(self, x, filters):
        x_skip = x
        x = Conv2D(filters, (3, 3), strides=(2, 2), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x = self.activation_layer(x)

        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)

        x_skip = Conv2D(filters, (1, 1), strides=(2, 2), kernel_regularizer=l2(1e-4))(x_skip)
        x = Add()([x, x_skip])
        x = self.activation_layer(x)
        return x

    def build(self):
        x_input = Input(self.input_shape)
        x = ZeroPadding2D((3, 3))(x_input)
        x = Conv2D(64, (7, 7), strides=2, padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization()(x)
        x = self.activation_layer(x)
        x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

        filter_sizes = [64, 128, 256, 512]
        block_layers = [2, 2, 2, 2]

        for i in range(4):
            filters = filter_sizes[i]
            if i == 0:
                for _ in range(block_layers[i]):
                    x = self.identity_block18(x, filters)
            else:
                x = self.convolutional_block18(x, filters)
                for _ in range(block_layers[i] - 1):
                    x = self.identity_block18(x, filters)

        x = AveragePooling2D((2, 2), padding='same')(x)
        x = Flatten()(x)
        x = Dropout(0.5)(x)
        x = Dense(self.num_classes, activation='softmax', kernel_regularizer=l2(1e-4))(x)

        return Model(inputs=x_input, outputs=x, name="ResNet18_TanhReLU")

# Build and compile model
rn = ResNet18(input_shape=(28, 28, 1))
model = rn.build()
model.summary()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

# Callbacks
earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train model
learning = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    verbose=1,
    callbacks=[earlystop, reduce_lr]
)

# Collect training history
loss_train = learning.history["loss"]
loss_val = learning.history["val_loss"]
acc_train = learning.history["sparse_categorical_accuracy"]
acc_val = learning.history["val_sparse_categorical_accuracy"]

# Evaluate model
y_pred = [np.argmax(pred) for pred in model.predict(X_test)]
prf_score = precision_recall_fscore_support(y_test, y_pred, average="weighted")
score = model.evaluate(X_test, y_test)

# Save performance
performance = {
    'loss_train': loss_train,
    'loss_val': loss_val,
    'acc_train': acc_train,
    'acc_val': acc_val,
    'prf_score': prf_score,
    'score': score
}

performance_file_path = '/content/drive/My Drive/Colab Notebooks/custom1_on_fmnist_performance.json'

Mounted at /content/drive
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "ResNet18_TanhReLU"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 34, 34, 1) │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 17, 17,    │      3,200 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 17, 17,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tanh_re_lu          │ (None, 2, 2, 512) │          0 │ batch_normalizat… │
│ (TanhReLU)          │                   │            │ batch_normalizat… │
│                     │                   │            │ add[0][0],        │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_1[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_2[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_3[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_4[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_5[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_6[0][0],      │
│                     │                   │            │ batch_normalizat… │
│                     │                   │            │ add_7[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 9, 9, 64)  │          0 │ tanh_re_lu[0][0]  │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 9, 9, 64)  │     36,928 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 64)  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 9, 9, 64)  │     36,928 │ tanh_re_lu[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 64)  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 9, 9, 64)  │          0 │ batch_normalizat… │
│                     │                   │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 9, 9, 64)  │     36,928 │ tanh_re_lu[2][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 64)  │        256 │ conv2d_3[0][0]  

 Total params: 11,186,186 (42.67 MB)

 Trainable params: 11,178,378 (42.64 MB)

 Non-trainable params: 7,808 (30.50 KB)

Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 71s 24ms/step - loss: 1.0557 - sparse_categorical_accuracy: 0.7802 - val_loss: 0.5741 - val_sparse_categorical_accuracy: 0.8706 - learning_rate: 0.0010
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step - loss: 0.5467 - sparse_categorical_accuracy: 0.8699 - val_loss: 0.5147 - val_sparse_categorical_accuracy: 0.8716 - learning_rate: 0.0010
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 37s 20ms/step - loss: 0.4826 - sparse_categorical_accuracy: 0.8816 - val_loss: 0.4996 - val_sparse_categorical_accuracy: 0.8791 - learning_rate: 0.0010
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - loss: 0.4691 - sparse_categorical_accuracy: 0.8874 - val_loss: 0.4948 - val_sparse_categorical_accuracy: 0.8874 - learning_rate: 0.0010
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 0.4537 - sparse_categorical_accuracy: 0.8946 - val_loss: 0.4678 - val_sparse_categorical_accuracy: 0.8891 - learning_rate: 0.0010
Epoch 6/30
1875/1875 ━━━━━━━━━

In [2]:
with open(performance_file_path, 'w') as f:
    json.dump(performance, f)

print("Performance saved to:", performance_file_path)

Performance saved to: /content/drive/My Drive/Colab Notebooks/custom1_on_fmnist_performance.json
